<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-06-05 10:41:50
-------------------
qualified stocks: 93
with latest results: 31
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  1.64 L
Current:  1.32 C
-------------------
Today PnL: 53.45 K (0.41%)
Current PnL: -16.16 L (-11.26%)
CY Booked + Current PnL: -11.17 L (-7.78%)
-------------------
Total profit:  4.15 L
Total loss:  -20.32 L
-------------------
Total Booked + Current PnL: 15.81 L (13.56%)
Total Booked PnL: 31.97 L (27.43%)
Curr Year Booked PnL: 4.99 L (3.77%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.12 C
Est FTT PnL: 79.12 L (59.73%)
-------------------
Est LTT:  2.74 C
Est LTT PnL: 1.41 C (106.68%)
Deployed:  1.17 C
Current:  1.32 C
CAGR/XIRR %: 9.11%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,785.05,983.0,-2.96,H-LC,89.25,237795.0,43103.0,5992.0,0.59,22.14,2.52,25.21,31.0,7.19,1.82,31.49,XY25,NTT,INSURANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104058.0,25696.0,8335.0,2.05,32.79,8.01,43.43,70.0,3.08,0.80,39.18,XY24,NTT,MISC
62,RELIANCE,1291.83,1526.0,4.61,X-LC,26.88,155162.0,14353.0,11172.0,1.28,10.19,7.20,18.13,19.0,1.28,1.19,22.51,XY25,NTT,REFINERIES
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133331.0,35684.0,11986.0,0.81,36.54,8.99,48.82,134.0,2.98,1.02,48.12,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,91.40,172290.0,31450.0,13042.0,-1.06,22.33,7.57,31.59,79.0,2.41,1.32,49.66,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SFL,1016.35,1287.0,287.26,H-SC,29.03,174924.0,-87294.0,157117.0,8.24,-33.29,89.82,26.63,136.0,-0.56,1.34,8.76,XY24,NTT,MISC
30,GREENPANEL,375.16,537.0,192.64,M-SC,80.65,130784.0,-52294.0,131268.0,2.49,-28.56,100.37,43.14,191.0,-0.40,1.00,23.02,XY24,NTT,MISC
27,DMART,3764.92,5201.0,-18.73,X-LC,46.24,204301.0,19820.0,50544.0,2.33,10.74,24.74,38.14,15.0,0.39,1.56,22.49,AR,NTT,FMCG
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104058.0,25696.0,8335.0,2.05,32.79,8.01,43.43,70.0,3.08,0.80,39.18,XY24,NTT,MISC
91,WHIRLPOOL,1167.49,2270.0,-35.67,M-SC,44.09,187425.0,15804.0,146266.0,1.91,9.21,78.04,94.43,177.0,0.11,1.43,38.33,X40,NTT,DURABLES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UNIONBANK,123.87,163.0,-1.84,M-LC,91.40,172290.0,31450.0,13042.0,-1.06,22.33,7.57,31.59,79.0,2.41,1.32,49.66,XY24,NTT,BANKS
41,INDUSINDBK,1354.50,1800.0,-37.72,H-MC,7.53,91884.0,-62529.0,113321.0,-1.03,-40.49,123.33,32.89,106.0,-0.55,0.70,26.52,XR,NTT,BANKS
53,MASFIN,326.60,399.5,-25.02,H-SC,58.06,85320.0,-12660.0,34529.0,-0.94,-12.92,40.47,22.32,127.0,-0.37,0.65,23.74,XR,ATH,FINANCE
24,DABUR,505.20,735.0,-12.22,X-MC,31.18,190281.0,-6747.0,96377.0,-0.80,-3.42,50.65,45.49,51.0,-0.07,1.45,8.09,XY24,BTT,FMCG
59,RAJESHEXPO,517.65,583.0,1864.65,M-SC,53.76,53596.0,-83581.0,100900.0,-0.78,-60.93,188.26,12.62,209.0,-0.83,0.41,33.30,X40N,NTT,JEWELLERY


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,1461.46,2275.00,-13.50,X-LC,50.54,277164.0,15563.0,130073.0,-0.18,5.95,46.93,55.67,1.0,0.12,2.12,12.42,X40,BTT,IT
80,TCS,3794.03,4998.00,-18.73,X-LC,20.43,264443.0,-31491.0,125399.0,0.28,-10.64,47.42,31.73,2.0,-0.25,2.02,5.85,X40,BTT,IT
51,LTIM,5564.16,7197.33,-3.95,X-LC,77.42,185094.0,-15216.0,74019.0,-0.22,-7.60,39.99,29.35,3.0,-0.21,1.41,27.87,X200,ATH,IT
7,ASIANPAINT,2961.56,4250.00,-23.26,X-LC,3.23,152701.0,-48685.0,136301.0,-0.15,-24.17,89.26,43.51,5.0,-0.36,1.17,5.32,X40,BTT,PAINTS
35,HINDUNILVR,2413.81,2723.00,-15.82,X-LC,25.81,264298.0,-6049.0,40675.0,-0.20,-2.24,15.39,12.81,6.0,-0.15,2.02,9.38,XY25,NTT,FMCG


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133331.0,35684.0,11986.0,0.81,36.54,8.99,48.82,134.0,2.98,1.02,48.12,X40,ATH,PHARMA
85,UNIONBANK,123.87,163.0,-1.84,M-LC,91.40,172290.0,31450.0,13042.0,-1.06,22.33,7.57,31.59,79.0,2.41,1.32,49.66,XY24,NTT,BANKS
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104058.0,25696.0,8335.0,2.05,32.79,8.01,43.43,70.0,3.08,0.80,39.18,XY24,NTT,MISC
50,LICI,785.05,983.0,-2.96,H-LC,89.25,237795.0,43103.0,5992.0,0.59,22.14,2.52,25.21,31.0,7.19,1.82,31.49,XY25,NTT,INSURANCE


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SAIL,130.64,228.00,101.81,M-MC,87.10,229422.0,4460.0,163188.0,1.06,1.98,71.13,74.53,170.0,0.03,1.75,33.20,XY24,BTT,STEEL
52,LUXIND,1448.19,2442.00,40.00,M-SC,69.89,145350.0,7772.0,86643.0,0.29,5.65,59.61,68.62,183.0,0.09,1.11,26.01,X40N,NTT,TEXTILES
56,PFIZER,4245.50,6318.30,15.43,H-SC,98.92,133331.0,35684.0,11986.0,0.81,36.54,8.99,48.82,134.0,2.98,1.02,48.12,X40,ATH,PHARMA
57,POLYCAB,5005.95,7541.35,6.25,H-MC,38.71,100861.0,15760.0,27343.0,0.61,18.52,27.11,50.65,90.0,0.58,0.77,26.79,X40N,ATH,CABLES
18,CAMS,3643.00,5250.99,4.67,H-SC,66.67,117628.0,15624.0,29395.0,0.66,15.32,24.99,44.14,112.0,0.53,0.90,34.05,XR,ATH,MISC


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,4.67,H-SC,66.67,117628.0,15624.0,29395.0,0.66,15.32,24.99,44.14,112.0,0.53,0.90,34.05,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-28.22,M-MC,59.14,141214.0,12062.0,74999.0,-0.64,9.34,53.11,67.42,156.0,0.16,1.08,37.34,XR,NTT,BANKS
39,INDIAMART,2327.09,4911.36,-51.28,H-SC,60.22,126723.0,3387.0,133579.0,0.47,2.75,105.41,111.05,108.0,0.03,0.97,24.17,AR,ATH,MISC
92,WIPRO,243.46,420.00,-10.62,M-LC,34.41,153444.0,2499.0,106950.0,0.26,1.66,69.70,72.51,68.0,0.02,1.17,11.81,XR,NTT,IT
1,AARTIIND,487.04,919.00,40.73,M-SC,61.29,80761.0,-575.0,72709.0,0.46,-0.71,90.03,88.69,205.0,-0.01,0.62,36.19,XR,NTT,CHEMICALS


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MASFIN,326.60,399.5,-25.02,H-SC,58.06,85320.0,-12660.0,34529.0,-0.94,-12.92,40.47,22.32,127.0,-0.37,0.65,23.74,XR,ATH,FINANCE
4,ADANIPORTS,1103.69,1583.0,3.60,M-LC,64.52,104058.0,25696.0,8335.0,2.05,32.79,8.01,43.43,70.0,3.08,0.80,39.18,XY24,NTT,MISC
87,VALIANTORG,512.64,838.0,-376.36,H-SC,100.00,117559.0,-15727.0,100325.0,1.19,-11.80,85.34,63.47,130.0,-0.16,0.90,97.06,XR,NTT,CHEMICALS
84,UJJIVANSFB,52.77,60.0,36.52,M-SC,56.99,118827.0,-23652.0,43170.0,1.45,-16.60,36.33,13.70,221.0,-0.55,0.91,40.25,X40N,NTT,BANKS
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133331.0,35684.0,11986.0,0.81,36.54,8.99,48.82,134.0,2.98,1.02,48.12,X40,ATH,PHARMA


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VALIANTORG,512.64,838.0,-376.36,H-SC,100.00,117559.0,-15727.0,100325.0,1.19,-11.80,85.34,63.47,130.0,-0.16,0.90,97.06,XR,NTT,CHEMICALS
56,PFIZER,4245.50,6318.3,15.43,H-SC,98.92,133331.0,35684.0,11986.0,0.81,36.54,8.99,48.82,134.0,2.98,1.02,48.12,X40,ATH,PHARMA
8,ASIANTILES,75.41,137.0,6550.00,M-SC,97.85,72948.0,-20862.0,97480.0,1.09,-22.24,133.63,81.67,214.0,-0.21,0.56,41.27,XR,NTT,CERAMICS
81,TITAGARH,1097.23,1548.0,-7.32,H-SC,96.77,162820.0,-29195.0,108080.0,0.04,-15.20,66.38,41.08,128.0,-0.27,1.24,37.81,XY24,NTT,ENGINEERING
74,STARHEALTH,551.56,761.0,4.66,M-SC,95.70,171209.0,-29007.0,105037.0,0.97,-14.49,61.35,37.97,195.0,-0.28,1.31,37.47,XY24,NTT,INSURANCE


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,42.05
2,50,70.49


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    31.70
AR      15.02
XY25    14.55
X40N    13.31
X40     12.50
XR      10.39
X200     1.41
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.12
X-LC    17.98
M-SC    17.49
H-LC     9.14
H-MC     8.50
M-LC     7.69
M-MC     5.53
X-MC     5.53
X-SC     2.77
L-SC     2.31
L-LC     1.12
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.42,-2.03,33.99
IT,10.17,-17.23,80.92
BANKS,9.67,-10.86,61.83
MISC,6.55,-26.27,82.12
FINANCE,5.83,-25.34,84.85
PAINTS,5.31,-18.37,53.39
INSURANCE,5.30,3.77,25.08
ELECTRICAL,5.12,-4.88,50.38
HEALTHCARE,4.37,-4.04,35.08


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2827183.0,26
AR,1261156.0,14
X40N,1207720.0,17
XR,1094640.0,14
X40,774347.0,9
XY25,474573.0,10
SR,198074.0,2
X200,74019.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2211406.0,20
M-SC,2031279.0,21
X-LC,859356.0,11
M-MC,544236.0,5
H-MC,514438.0,8
X-SC,346457.0,4
X-MC,308206.0,5
M-LC,299752.0,6
L-SC,295814.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      922386.0      6
M-SC       XY24      751431.0      6
H-SC       AR        541727.0      3
           X40N      497440.0      6
X-LC       X40       480843.0      4
M-SC       X40N      471371.0      6
M-MC       XY24      387407.0      3
M-SC       XR        289179.0      4
H-LC       AR        250743.0      5
H-SC       XR        237867.0      4
M-SC       X40       218788.0      2
M-LC       XY24      177462.0      4
H-MC       XY24      170466.0      2
X-LC       AR        167231.0      2
L-SC       XR        156884.0      2
X-MC       XY24      139311.0      2
X-LC       XY25      137263.0      4
M-SC       XY25      121246.0      1
X-SC       X40N      118033.0      2
           XR        115440.0      1
H-MC       XR        113321.0      1
X-SC       SR        112984.0      1
M-LC       XR        106950.0      1
M-SC       AR         94174.0      1
X-MC       X40N       93533.0      2
M-SC       SR         85090.0      1
M-MC       AR         81830.0      1
L-SC       XY24       77775.0      1
H-MC       XY25       76282.0      1
X-MC       XY25       75362.0      1
M-MC       XR         74999.0      1
X-LC       X200       74019.0      1
H-MC       AR         64296.0      1
           X40        62730.0      2
L-SC       AR         61155.0      1
L-LC       XY25       43088.0      1
H-MC       X40N       27343.0      1
M-LC       XY25       15340.0      1
H-SC       X40        11986.0      1
H-LC       XY25        5992.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
